<p><strong><font size="6">WalOUS project</font></strong></p>

<p><strong><font size="6">B_Importation_Postgresql</font></strong></p>

WALOUS_LU - Copyright (C) <2020> <Université catholique de Louvain (UCLouvain), Belgique
					 Université Libre de Bruxelles (ULB), Belgique
					 Institut Scientifique de Service Public (ISSeP), Belgique
					 Service Public de Wallonie (SWP), Belgique >
						 							
	
List of the contributors to the development of WALOUS_LU: see LICENSE file.
Description and complete License: see LICENSE file.
	
This program (WALOUS_LU) is free software:
you can redistribute it and/or modify it under the terms of the
GNU General Public License as published by the Free Software
Foundation, either version 3 of the License, or (at your option)
any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program (see COPYING file).  If not,
see <http://www.gnu.org/licenses/>.

------- 
Jupyter Notebook containing the preprocessing steps consisting of: 
- Importation of source data in the PostgreSQL database.
- Cleaning some data. 
- When needed, transforming the actual input data labels to a HILUCS compatible label by using lookup tables. 

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

# Define working environment

**Import libraries**

In [ ]:
# Import libraries needed for setting parameters of operating system 
import os
import sys

**Add folder with SCR provided belong to this notebook**

In [ ]:
# Add local module to the path
src = os.path.abspath('../SRC')
if src not in sys.path:
    sys.path.append(src)

**Setup environment variables**

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [ ]:
run ../SRC/config.py

In [ ]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [ ]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

**Custom functions: Psycopg2 and Postgresql functions**

In [ ]:
# Import function that display postgresql's table header
from display_header import display_header
# Import function to creation connection to Postgresql database 
from postgres_functions import create_pg_connexion
# Import function to creation of Postgresql schema 
from postgres_functions import create_pg_schema
# Import function to give rights to user on a specific schema
from postgres_functions import grant_user

In [ ]:
# Import function that manage importation of a Shapefile into postgresql database
from postgres_import import shp2pgsql
from postgres_import import ogr2ogr_import
# Import a function that manage importation of a CSV file
from postgres_import import import_csv

In [ ]:
# Import function to create index
from postgres_functions import create_index
# Import function that fix not valid geometries
from postgres_geom import make_valid
# Import function that split multi-part polygon in single part polgons
from postgres_geom import split_multipolygon_to_singlepolygon

**Custom functions: others**

In [ ]:
# Import functions for processing time information
import time
from processing_time import start_processing, print_processing_time

# Cadastral informations

## Create new schema

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'agdp', overwrite=False)
grant_user(con, 'agdp', 'bbeaumont')
# Close connexion to postgres database
con.close()

## Import cadastral parcels

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['capa'][1], config_parameters, 
               schema='agdp', table=data['capa'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'agdp', data['capa'][0], 'geom', geometry_type=3)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'agdp', data['capa'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'agdp', data['capa'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

### Remove rows where capakey = 'DP'

Cause issues because the capakey is not unique and could not be used as primary key in joins

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create cursor
cursor = con.cursor()
# Find how many rows will be removed
query="SELECT count(*) FROM %s.%s WHERE capakey = 'DP' GROUP BY capakey;"%('agdp', data['capa'][0])
cursor.execute(query)
count = cursor.fetchone()
if count:
    print ('%s row(s) will be removed from the table...'%count[0])
    # Delete from query
    query="DELETE FROM %s.%s WHERE capakey = 'DP';"%('agdp', data['capa'][0])
    cursor.execute(query)
    # Make the changes to the database persistent
    con.commit()
else:
    print ('Nothing to remove!')
# Close connexion to postgres database
con.close()

## Import cadastral matrix

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Import CSV file
import_csv(con, data['cama'][1], data['cama_type'], 'agdp', data['cama'][0], delimiter=';', overwrite=False)
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'agdp',data['cama'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## Import correspondance between 'nature cadastrale' and HILUCS LU legend

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Column declaration
columns_declaration = [('nature','integer'),('hilucs','text')]
# Import CSV file
import_csv(con, data['leg_capa'][1], columns_declaration, 'agdp', data['leg_capa'][0], delimiter=';', overwrite=False)
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'agdp', data['leg_capa'][0], row_num=8)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## Create a table with all HILUCS legends associated with each capakey

One capakey can have multiple "nature". Here, all the natures are converted into HILUCS classes and then aggregated in an array sorted from the most frequent to the last frequent HILUCS class in the parcel. This information is stored in column 'nat_lu_rank'. The column 'nat_nb_dist_lu' indicate of many distinct HILUCS classes appear in the cadastral parcel. The column 'nat_lu_maj' contains the HILUCS class that appear most frequently in the cadastral parcel and the frequence is given in column 'nat_lu_maj_count'.

In [ ]:
## Saving current time for processing time management
begintime_copy=time.time()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create cursor
cursor = con.cursor()
# Drop table if exits
cursor.execute("DROP TABLE IF EXISTS agdp.cama_hilucs_count;")
# Query 
mainquery = "SELECT a.capakey, b.hilucs, count(*) AS nb, rank() OVER (PARTITION BY capakey ORDER BY count(*) DESC) \
FROM agdp.cama AS a \
LEFT JOIN agdp.leg_capa AS b ON a.nature=b.nature \
GROUP BY capakey, hilucs"
# Create table query
cursor.execute("CREATE TABLE agdp.cama_hilucs_count AS (%s);"%mainquery)
# Make the changes to the database persistent
con.commit()
# Close connection with database
cursor.close()
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create cursor
cursor = con.cursor()
# Query 
cursor.execute("CREATE INDEX IF NOT EXISTS capakey_idx ON agdp.cama_hilucs_count (capakey);")
cursor.execute("CREATE INDEX IF NOT EXISTS hilucs_idx ON agdp.cama_hilucs_count (hilucs);")
cursor.execute("CREATE INDEX IF NOT EXISTS capakeyhilucs_idx ON agdp.cama_hilucs_count (capakey,hilucs);")
# Make the changes to the database persistent
con.commit()
# Close connection with database
cursor.close()
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create cursor
cursor = con.cursor()
# Drop table if exits
cursor.execute("DROP TABLE IF EXISTS agdp.cama_hilucs_array;")
# Query 
mainquery = "SELECT capakey AS capakey_cama, \
(SELECT hilucs FROM agdp.cama_hilucs_count b WHERE b.capakey=a.capakey ORDER BY rank LIMIT 1) AS nat_lu_maj, \
(SELECT nb FROM agdp.cama_hilucs_count b WHERE b.capakey=a.capakey ORDER BY rank LIMIT 1) AS nat_lu_maj_count, \
array_length(array_agg(a.hilucs),1) AS nat_nb_dist_lu, \
array_agg(a.hilucs ORDER BY a.rank) AS nat_lu_rank \
FROM agdp.cama_hilucs_count AS a \
GROUP BY capakey"
# Create table query
cursor.execute("CREATE TABLE agdp.cama_hilucs_array AS (%s);"%mainquery)
# Make the changes to the database persistent
con.commit()
# Close connection with database
cursor.close()
# Close connexion to postgres database
con.close()

In [ ]:
## Compute processing time and print it
print(print_processing_time(begintime_copy, "Process achieved in "))

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'agdp', 'cama_hilucs_array', where='nat_nb_dist_lu > 1', row_num=15)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# DBRIS - Economic activites

## Create new schema

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'dbris', overwrite=False)
grant_user(con, 'dbris', 'bbeaumont')
# Close connexion to postgres database
con.close()

## Import table of correspondance between NACE code and HILUCS LU legend

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Column declaration
columns_declaration = [('nace','text'),('nace_label','text'),('hilucs_l2','text'),
                       ('hilucs_l2_label','text'),('hilucs_mix','text'),('hilucs_mix_label','text'), 
                       ('hilucs_walous','text'),('hilucs_walous_label','text')]
# Import CSV file
import_csv(con, data['leg_nace'][1], columns_declaration, 'dbris', data['leg_nace'][0], 
           delimiter='|', overwrite=False)
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'dbris', data['leg_nace'][0], row_num=500)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## DBRIS Table 1 (main nace per capakey and number of establishments)

### Import table (csv)

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Import CSV file
import_csv(con, data['dbris1'][1], data['dbris1_type'], 'dbris', data['dbris1'][0], delimiter=',', overwrite=False)
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'dbris',data['dbris1'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

### Transform nace 3 digits to nace 2 digits

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create cursor
cursor = con.cursor()
# Update table to have proper 2-digits nace code for code not starting with a 01 
# i.e., nace 015 remains 015, but nace 024 become 02 and nace 823 become 82
query = "UPDATE %s.%s SET nace_main = left(nace_main,2) WHERE left(nace_main,2) != '01';"%('dbris', data['dbris1'][0])
cursor.execute(query)
# Make the changes to the database persistent
con.commit()
# Close connection with database
cursor.close()
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'dbris',data['dbris1'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

### Create a table NACE code translated into the WalOUS HILUCS legends

In [ ]:
## Saving current time for processing time management
begintime_copy=time.time()
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create cursor
cursor = con.cursor()
# Drop table if exits
cursor.execute("DROP TABLE IF EXISTS %s.%s_hilucs;"%('dbris', data['dbris1'][0]))
# Query 
mainquery = "SELECT a.capakey AS capakey_%s, \
b.hilucs_walous as dbris_maj, a.nace_main_share as dbris_maj_share, \
a.count_etab as dbris_count_etab "%data['dbris1'][0]
mainquery += "FROM %s.%s AS a "%('dbris', data['dbris1'][0])
mainquery += "LEFT JOIN %s.%s AS b ON a.nace_main=b.nace "%('dbris', data['leg_nace'][0])

# Create table query
cursor.execute("CREATE TABLE %s.%s_hilucs AS (%s);"%('dbris', data['dbris1'][0],mainquery))

# Make the changes to the database persistent
con.commit()
# Close connection with database
cursor.close()
# Close connexion to postgres database
con.close()
## Compute processing time and print it
print(print_processing_time(begintime_copy, "Process achieved in "))

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'dbris', '%s_hilucs'%data['dbris1'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## DBRIS Table 2 (ranking of nace per capakey and number of establishments per quality of geocoding)

### Import table (csv)

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Import CSV file
import_csv(con, data['dbris2'][1], data['dbris2_type'], 'dbris', data['dbris2'][0], 
          delimiter=',', add_serial_primary_key=True, overwrite=False)
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'dbris',data['dbris2'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

### Transform 3-digits "nace" to nace 2-digits "nace"

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create cursor
cursor = con.cursor()
# Update table to have proper 2-digits nace code for code not starting with a 01 
# i.e., nace 015 remains 015, but nace 024 become 02 and nace 823 become 82
query = "UPDATE %s.%s SET cd_nace = left(cd_nace,2) WHERE left(cd_nace,2) != '01';"%('dbris', data['dbris2'][0])
cursor.execute(query)
# Make the changes to the database persistent
con.commit()
# Close connection with database
cursor.close()
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'dbris',data['dbris2'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

### Create a table with array containing distinct ordered HILUCS classes from DBRIS database and quality of geocoding

"dbris_rank" will contain distinct hilucs activities in the corresponding cadastral plot, ranked according to the relative importance of employees per 3-digits "nace" activity. 

"dbris_accur_geocod" takes value True only if the all establishment got an accurate geocoding and the assigned coordinates felt within a cadastral parcel.  

In [ ]:
## Saving current time for processing time management
begintime_copy=time.time()
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create cursor
cursor = con.cursor()
# Drop table if exits
cursor.execute("DROP TABLE IF EXISTS %s.%s_hilucs;"%('dbris', data['dbris2'][0]))
# With query 
withquery = "WITH a AS ("
withquery += "SELECT a.capakey AS capakey_%s, \
b.hilucs_walous as dbris_hilucs, a.ms_rank as dbris_hilucs_rank, \
CASE WHEN a.ms_cnt_etab_accurate = a.ms_cnt_etab AND a.ms_cnt_etab_pip = a.ms_cnt_etab \
THEN True ELSE False END as dbris_accur_geocod "%data['dbris2'][0]
withquery += "FROM %s.%s AS a "%('dbris', data['dbris2'][0])
withquery += "LEFT JOIN %s.%s AS b ON a.cd_nace=b.nace), "%('dbris', data['leg_nace'][0])
withquery += "b AS (SELECT DISTINCT capakey_dbris2, dbris_hilucs, \
min(dbris_hilucs_rank) AS dbris_hilucs_rank, bool_and(dbris_accur_geocod) as dbris_accur_geocod \
FROM a GROUP BY capakey_dbris2, dbris_hilucs ORDER BY min(dbris_hilucs_rank))"
# Main query 
mainquery = withquery
mainquery += "SELECT capakey_dbris2, ARRAY_AGG(dbris_hilucs) as dbris_rank, \
bool_and(dbris_accur_geocod) as dbris_accur_geocod FROM b GROUP BY capakey_dbris2"
# Create table query
cursor.execute("CREATE TABLE %s.%s_hilucs AS (%s);"%('dbris', data['dbris2'][0],mainquery))
# Make the changes to the database persistent
con.commit()
# Close connection with database
cursor.close()
# Close connexion to postgres database
con.close()
## Compute processing time and print it
print(print_processing_time(begintime_copy, "Process achieved in "))

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'dbris', '%s_hilucs'%data['dbris2'][0], row_num=15)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# Percentage of LC class in cadastral blocs

## Import csv output from r.zonal.classes

In [ ]:
# Create a list for the column declaration of the CREATE table statement
with open(data['lc_capa'][1], 'r') as f:
    header = next(f).split("\n")[0]
colum_declaration = [("capakey_lc_prop",'varchar') if col == "CaPaKey" else ("lc_mode",'integer') if col == "lc_mode" else (col,'double precision') for col in header.split("|")]# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Import CSV file
import_csv(con, data['lc_capa'][1], colum_declaration, 'agdp', data['lc_capa'][0], delimiter="|", overwrite=False)
# Close connexion to postgres database
con.close()

In [ ]:
# Remove 'cat' column
con = create_pg_connexion(config_parameters)
cursor = con.cursor()
cursor.execute("ALTER TABLE agdp.%s DROP COLUMN IF EXISTS cat;"%data['lc_capa'][0])
con.commit()
cursor.close()
con.close()

In [ ]:
# Add index on 'capakey'
con = create_pg_connexion(config_parameters)
create_index(con, 'agdp', data['lc_capa'][0], 'capakey_lc_prop', is_geom=False)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'agdp', data['lc_capa'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# Population data

**Create new schema**

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'rnpp', overwrite=False)
grant_user(con, 'sar', 'bbeaumont')
# Close connexion to postgres database
con.close()

## Point data

**Import layer**

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['rnpp'][1], config_parameters, 
               schema='rnpp', table=data['rnpp'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'rnpp', data['rnpp'][0], 'geom', geometry_type=1)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'rnpp', data['rnpp'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'rnpp', data['rnpp'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## Neighborhood density data

**Import csv output from r.zonal.classes**

In [ ]:
# Create a list for the column declaration of the CREATE table statement
with open(data['rnpp_neighbor'][1], 'r') as f:
    header = next(f).split("\n")[0]
colum_declaration = [("capakey_%s"%data['rnpp_neighbor'][0],'varchar') if col == "CaPaKey" else (col,'integer') for col in header.split("|")]
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Import CSV file
import_csv(con, data['rnpp_neighbor'][1], colum_declaration, 'rnpp', data['rnpp_neighbor'][0], delimiter="|", overwrite=False)
# Close connexion to postgres database
con.close()

In [ ]:
# Remove 'cat' column
con = create_pg_connexion(config_parameters)
cursor = con.cursor()
cursor.execute("ALTER TABLE rnpp.%s DROP COLUMN IF EXISTS cat;"%data['rnpp_neighbor'][0])
con.commit()
cursor.close()
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'rnpp', data['rnpp_neighbor'][0], "capakey_%s"%data['rnpp_neighbor'][0], is_geom=False)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'rnpp', data['rnpp_neighbor'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# Agricultural lands (sigec)

**Create new schema**

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'sigec', overwrite=False)
grant_user(con, 'sar', 'bbeaumont')
# Close connexion to postgres database
con.close()

## Grasslands (1_1_1_A)

**Import layer**

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['sigec_p'][1], config_parameters, 
               schema='sigec', table=data['sigec_p'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'sigec', data['sigec_p'][0], 'geom', geometry_type=3)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'sigec', data['sigec_p'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'sigec', data['sigec_p'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## Arable lands (1_1_1_B)

**Import layer**

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['sigec_ta'][1], config_parameters, 
               schema='sigec', table=data['sigec_ta'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'sigec', data['sigec_ta'][0], 'geom', geometry_type=3)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'sigec', data['sigec_ta'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'sigec', data['sigec_ta'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# Brown fields (Former industrial area - SAR)

**Create new schema**

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'sar', overwrite=False)
grant_user(con, 'sar', 'bbeaumont')
# Close connexion to postgres database
con.close()

**Import layer**

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['sar'][1], config_parameters, 
               schema='sar', table=data['sar'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'sar', data['sar'][0], 'geom', geometry_type=3)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'sar', data['sar'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'sar', data['sar'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# Public forestry

**Create new schema**

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'forest', overwrite=False)
grant_user(con, 'forest', 'bbeaumont')
# Close connexion to postgres database
con.close()

**Import layer**

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['forestry_public'][1], config_parameters, 
               schema='forest', table=data['forestry_public'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'forest', data['forestry_public'][0], 'geom', geometry_type=3)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'forest', data['forestry_public'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'forest', data['forestry_public'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# Nature conservation

**Create new schema**

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'nat_conser', overwrite=False)
grant_user(con, 'nat_conser', 'bbeaumont')
# Close connexion to postgres database
con.close()

## Natura 2000

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['nature_conservation_n2000'][1], config_parameters, 
               schema='nat_conser', table=data['nature_conservation_n2000'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'nat_conser', data['nature_conservation_n2000'][0], 'geom', geometry_type=3)
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Split multipolygons into single part polygons
split_multipolygon_to_singlepolygon(con, 'nat_conser', data['nature_conservation_n2000'][0], 'ogc_fid', 'geom')
# Update geometry column to ensure its a valid geometry
make_valid(con, 'nat_conser', data['nature_conservation_n2000'][0], 'geom', geometry_type=3)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'nat_conser', data['nature_conservation_n2000'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'nat_conser', data['nature_conservation_n2000'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## Nature conservation class 7

The layer imported here has been created as a preprocessing step by merging shapefiles of "réserve naturelle agréée" and "réserve naturelle domaniale" and "Zones militaires (Camp Roi Albert, Camp Elsenborn Lager, Camp Lagland).

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['nature_conservation_7'][1], config_parameters, 
               schema='nat_conser', table=data['nature_conservation_7'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'nat_conser', data['nature_conservation_7'][0], 'geom', geometry_type=3)
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Split multipolygons into single part polygons
split_multipolygon_to_singlepolygon(con, 'nat_conser', data['nature_conservation_7'][0], 'ogc_fid', 'geom')
# Update geometry column to ensure its a valid geometry
make_valid(con, 'nat_conser', data['nature_conservation_7'][0], 'geom', geometry_type=3)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'nat_conser', data['nature_conservation_7'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'nat_conser', data['nature_conservation_7'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## Nature conservation class 7_1

The layer imported here has been created as a preprocessing step by merging shapefiles of "zone humide d'intérêt biologique (ZHIB)" and "réserve forestière". 

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['nature_conservation_71'][1], config_parameters, 
               schema='nat_conser', table=data['nature_conservation_71'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'nat_conser', data['nature_conservation_71'][0], 'geom', geometry_type=3)
# Close connexion to postgres database
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Split multipolygons into single part polygons
split_multipolygon_to_singlepolygon(con, 'nat_conser', data['nature_conservation_7'][0], 'ogc_fid', 'geom')
# Update geometry column to ensure its a valid geometry
make_valid(con, 'nat_conser', data['nature_conservation_71'][0], 'geom', geometry_type=3)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'nat_conser', data['nature_conservation_71'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'nat_conser', data['nature_conservation_71'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# Schools

**Create new schema**

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'schools', overwrite=False)
grant_user(con, 'schools', 'bbeaumont')
# Close connexion to postgres database
con.close()

**Import layer**

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['schools'][1], config_parameters, 
               schema='schools', table=data['schools'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'schools', data['schools'][0], 'geom', geometry_type=3)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'schools', data['schools'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'schools', data['schools'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# PICC surface polygons

**Create new schema**

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'picc', overwrite=False)
grant_user(con, 'picc', 'bbeaumont')
# Close connexion to postgres database
con.close()

**Import layer**

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['picc_surface'][1], config_parameters, 
               schema='picc', table=data['picc_surface'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'picc', data['picc_surface'][0], 'geom', geometry_type=3)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'picc', data['picc_surface'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'picc', data['picc_surface'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# PICC symbology

**Create new schema**

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'picc', overwrite=False)
grant_user(con, 'picc', 'bbeaumont')
# Close connexion to postgres database
con.close()

**Import layer**

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['picc_symbology'][1], config_parameters, 
               schema='picc', table=data['picc_symbology'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'picc', data['picc_symbology'][0], 'geom', geometry_type=1)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'picc', data['picc_symbology'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'picc', data['picc_symbology'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# Eoliennes

**Create new schema**

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'eoliennes', overwrite=False)
grant_user(con, 'eoliennes', 'bbeaumont')
# Close connexion to postgres database
con.close()

**Import layer**

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['eoliennes'][1], config_parameters, 
               schema='eoliennes', table=data['eoliennes'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'eoliennes', data['eoliennes'][0], 'geom', geometry_type=1)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'eoliennes', data['eoliennes'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'eoliennes', data['eoliennes'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# Recypark

**Create new schema**

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'recypark', overwrite=False)
grant_user(con, 'recypark', 'bbeaumont')
# Close connexion to postgres database
con.close()

**Import layer**

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['recypark'][1], config_parameters, 
               schema='recypark', table=data['recypark'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'recypark', data['recypark'][0], 'geom', geometry_type=1)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'recypark', data['recypark'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'recypark', data['recypark'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# Etablissements pour ainés

**Create new schema**

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'etab_aines', overwrite=False)
grant_user(con, 'etab_aines', 'bbeaumont')
# Close connexion to postgres database
con.close()

**Import layer**

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['etab_aines'][1], config_parameters, 
               schema='etab_aines', table=data['etab_aines'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'etab_aines', data['etab_aines'][0], 'geom', geometry_type=1)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'etab_aines', data['etab_aines'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'etab_aines', data['etab_aines'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# SEVESO

**Create new schema**

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'seveso', overwrite=False)
grant_user(con, 'seveso', 'bbeaumont')
# Close connexion to postgres database
con.close()

**Import layer**

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['seveso'][1], config_parameters, 
               schema='seveso', table=data['seveso'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'seveso', data['seveso'][0], 'geom', geometry_type=1)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'seveso', data['seveso'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'seveso', data['seveso'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# Campings

**Create new schema**

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'camping', overwrite=False)
grant_user(con, 'camping', 'bbeaumont')
# Close connexion to postgres database
con.close()

**Import layer**

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['camping'][1], config_parameters, 
               schema='camping', table=data['camping'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'camping', data['camping'][0], 'geom', geometry_type=1)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'camping', data['camping'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'camping', data['camping'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# IGN data - Airports, AmusementParks, AnimalParks, Carry, Sports

**Create new schema**

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'ign', overwrite=False)
grant_user(con, 'ign', 'bbeaumont')
# Close connexion to postgres database
con.close()

## Airports

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['aeroport'][1], config_parameters, 
               schema='ign', table=data['aeroport'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'ign', data['aeroport'][0], 'geom', geometry_type=1)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'ign', data['aeroport'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'ign', data['aeroport'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## AmusementParks

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['parc_lois'][1], config_parameters, 
               schema='ign', table=data['parc_lois'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'ign', data['parc_lois'][0], 'geom', geometry_type=1)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'ign', data['parc_lois'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'ign', data['parc_lois'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## AnimalParks

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['parc_anim'][1], config_parameters, 
               schema='ign', table=data['parc_anim'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'ign', data['parc_anim'][0], 'geom', geometry_type=1)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'ign', data['parc_anim'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'ign', data['parc_anim'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## Carry

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['carrier_ign'][1], config_parameters, 
               schema='ign', table=data['carrier_ign'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'ign', data['carrier_ign'][0], 'geom', geometry_type=1)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'ign', data['carrier_ign'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'ign', data['carrier_ign'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## Sports

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['ter_sport'][1], config_parameters, 
               schema='ign', table=data['ter_sport'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'ign', data['ter_sport'][0], 'geom', geometry_type=1)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'ign', data['ter_sport'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'ign', data['ter_sport'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

# Other data from SPW

**Create new schema**

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create new schema
create_pg_schema(con, 'spw', overwrite=False)
grant_user(con, 'spw', 'bbeaumont')
# Close connexion to postgres database
con.close()

## Carry

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['carrier_spw'][1], config_parameters, 
               schema='spw', table=data['carrier_spw'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'spw', data['carrier_spw'][0], 'geom', geometry_type=1)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'spw', data['carrier_spw'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'spw', data['carrier_spw'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## Christmas Trees

In [ ]:
# Import shapefile into postgis database
ogr2ogr_import(data['sapin_noel'][1], config_parameters, 
               schema='spw', table=data['sapin_noel'][0],
               to_srid='31370', create_opt='-overwrite') 

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Update geometry column to ensure its a valid geometry
make_valid(con, 'spw', data['sapin_noel'][0], 'geom', geometry_type=3)
# Close connexion to postgres database
con.close()

In [ ]:
# Add index on 'geom'
con = create_pg_connexion(config_parameters)
create_index(con, 'spw', data['sapin_noel'][0], 'geom', is_geom=True)
con.close()

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
df = display_header(con, 'spw', data['sapin_noel'][0], row_num=5)
# Close connexion to postgres database
con.close()
# Display dataframe
df